In [21]:
import pandas as pd
import plotly.express as px
import numpy as np
import os

segment = 'method' # 'contract' # 'project'

In [22]:
prepend = 'csv_exports/whale_actions/'

op_df = pd.read_csv(prepend + 'op_whale_actions_22_01_18.csv')
op_df['chain'] = 'Optimism'

l1_df = pd.read_csv(prepend + 'l1_whale_actions_22_01_18.csv')
l1_df['chain'] = 'Ethereum'

arb_df = pd.read_csv(prepend + 'arbi_whale_actions_22_01_18.csv')
arb_df['chain'] = 'Arbitrum'

df = pd.concat([op_df, l1_df, arb_df])
df.sample(20)

,address,token_direction,project,contract,method,usd_value,num_tfers,num_txs,chain
107959,0x8764c5ddea10f9ddac33e00ad05f718e688ce394,in,0xb8aea6e724c8a5c3be46ad71ae5701d413e1d490,NaN,transfer,2.432033e+06,3,3,Ethereum
522,0xa0e1e0e6272be473db91bcba4fb4aac24affcacc,out,OTHER,OTHER,OTHER,4.086464e+01,2,2,Ethereum
275304,0xbda23b750dd04f792ad365b5f2a6f1d8593796f2,in,0xf434a2fd2beab1fd454f16d43db44742822f8a76,NaN,transfer,1.007082e+04,1,1,Ethereum
55797,0x9a9792d41c4e3816ed62e34be9b809d9190df94c,out,0x6b853d74c79c5a442a647dd902c4bfa16a23afac,NaN,transfer,1.139413e+04,1,1,Ethereum
212070,0x06d3a30cbb00660b85a30988d197b1c282c6dcb6,in,0xb29de7a42afeb1da549d640dafbd4cf414fe2af3,NaN,transfer,1.796883e+04,8,8,Ethereum
197720,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,out,0x64c483c7f6459e5eddc9d26e7cacad7fee5d61d0,NaN,0x,1.959795e+04,2,2,Ethereum
265334,0x58dfcf135d38aa376bcf881f5d2099acf81b7281,out,0xbda8858b22abe90bf53612e81b8a223a12c71f9b,NaN,transfer,5.579234e+04,1,1,Ethereum
17635,0xd952beecffc6bc1228e83d0457d25d5a39555abf,out,0x132f0546ae324108f0016b737054647afcb58b50,NaN,transfer,8.007263e+04,2,2,Ethereum
267964,0xa8779ce7e0d4f21e876fd8deb21f30d8ff4e119b,out,0xdeba44b0553499a931b56896fcccfdcae5d3b12e,NaN,transfer,3.907478e+04,1,1,Ethereum
176614,0x5b5ecfc8122ba166b21d6ea26268ef97e09b2e9f,in,0x2761a575cf6ea58d9d59426f1733df573d92cf36,NaN,transfer,1.004344e+04,2,2,Ethereum


In [23]:

df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [24]:
df_addr = df.groupby(['chain','address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby('address')[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/1126789040.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_addr = df.groupby(['chain','address','project','contract','method']).sum()


In [25]:
group_list = ['chain','project','contract','method']
dlist = ['chain','project','contract','method','action',\
                'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share','address']

if segment == 'project':
        group_list.drop('method',inplace=True)
        group_list.drop('contract',inplace=True)
        dlist.drop('method',inplace=True)
        dlist.drop('contract',inplace=True)
elif segment == 'contract':
        group_list.drop('method',inplace=True)
        dlist.drop('method',inplace=True)

In [26]:
df_dollar = df[['chain','project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['chain','project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [27]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['chain','project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'address':'nunique'
})
df_action.name = 'df_action'

# display(df_action)

In [40]:
dfs = [df_action, df_dollar]


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        if segment == 'method':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        elif segment == 'contract':
                d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str)
        elif segment == 'project':
                d['action'] = d['project'].astype(str)
        
        d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')

# df_action.sample(20)
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')

,chain,project,contract,method,action,usd_value_pct_share,usd_value_in_pct_share,usd_value_out_pct_share,usd_value_net_pct_share,address
308,Arbitrum,uniswap_v3,Pair,0x00271001,uniswap_v3 - Pair | 0x00271001,0.000761,0.001151,0.000000,0.001151,2
1746,Ethereum,ribbon,RibbonEarnVault,completeWithdraw,ribbon - RibbonEarnVault | completeWithdraw,0.243067,0.419613,0.000000,0.419613,9
1896,Ethereum,sushi,Pair,swapSimpleMode,sushi - Pair | swapSimpleMode,0.001993,0.000000,0.003399,0.000000,1
366,Arbitrum,uniswap_v3,Pair,0x021d0401,uniswap_v3 - Pair | 0x021d0401,0.000168,0.000264,0.000000,0.000264,1
1893,Ethereum,sushi,Pair,swapExactTokensForETH,sushi - Pair | swapExactTokensForETH,0.090580,0.000000,0.235587,0.000000,21
2261,Optimism,Perpetual Protocol,Vault,withdraw,Perpetual Protocol - Vault | withdraw,0.153932,0.308902,0.000000,0.308902,4
1949,Ethereum,thegraph,Staking,stake,thegraph - Staking | stake,1.000000,NaN,1.000000,NaN,1
704,Ethereum,aave_v2,WETHGateway,borrowETH,aave_v2 - WETHGateway | borrowETH,0.017535,0.036375,0.000000,0.036375,2
790,Ethereum,arbitrum,Bridge,executeTransaction,arbitrum - Bridge | executeTransaction,0.220897,0.277155,0.000000,0.277155,11
1778,Ethereum,seaport,Seaport,batchBuyWithETH,seaport - Seaport | batchBuyWithETH,0.025326,0.054176,0.000000,0.054176,5


In [39]:
print('Dollar-Weighted')
chains = df_dollar['chain'].drop_duplicates().to_list()
# print('Top Dollar Weighted Actions by Chain')

for c in chains:
        tdf = df_dollar[df_dollar['chain'] == c]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).sum()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )

Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/196019526.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
672,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInputSingle,0.908970,0.136786
671,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactInput,0.544242,0.071865
37,Arbitrum,celer_network,celer_network - Bridge | addLiquidity,0.525326,0.000000
673,Arbitrum,uniswap_v3,uniswap_v3 - Pair | exactOutput,0.471929,0.082285
125,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,0.381631,0.000000
678,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,0.272238,0.050338
130,Arbitrum,gmx,gmx - Vault | 0x364e2311,0.217463,0.000000
1,Arbitrum,OTHER,OTHER - OTHER | OTHER,0.216843,0.034668
136,Arbitrum,gmx,gmx - Vault | swap,0.142702,0.000106
8,Arbitrum,across_v2,across_v2 - Arbitrum_SpokePool | deposit,0.133860,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/196019526.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


...

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/196019526.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).sum()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
23,Optimism,Celer,Celer - Bridge | addLiquidity,1.002681,0.000000
263,Optimism,Uniswap,Uniswap - Pool | 0x608d60eb,0.883819,0.134040
269,Optimism,Uniswap,Uniswap - Pool | exactInputSingle,0.200170,0.031566
69,Optimism,Kyberswap,Kyberswap - AntiSnipAttackPositionManager | mu...,0.156633,0.041367
26,Optimism,Celer,Celer - Bridge | send,0.103441,0.000000
74,Optimism,Kyberswap,Kyberswap - Elastic_Pool | multicall,0.102830,0.000000
96,Optimism,OTHER,OTHER - OTHER | OTHER,0.090966,0.018323
104,Optimism,Perpetual Protocol,Perpetual Protocol - Vault | depositFor,0.074724,0.000000
290,Optimism,Velodrome,Velodrome - Pair | addLiquidity,0.072420,0.000000
4,Optimism,Aave,Aave - AToken | repay,0.069251,0.000000


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/196019526.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100
38,Optimism,Uniswap,1.314341,0.215869
7,Optimism,Celer,1.148165,0.205179
17,Optimism,Kyberswap,0.292208,0.044174
1,Optimism,Aave,0.146573,0.035010
39,Optimism,Uniswap V3,0.139361,0.028245
34,Optimism,Sonne Finance,0.137208,0.023588
40,Optimism,Velodrome,0.094357,0.012420
22,Optimism,OTHER,0.090966,0.018323
26,Optimism,Perpetual Protocol,0.087592,0.013292
9,Optimism,Curve,0.078605,0.008993


In [44]:
print('Address-Weighted')
for c in chains:
        tdf = df_action[df_action['chain'] == c]
        tdf = tdf[tdf['address'] >=20]
        sumdf = []
        print('Sum by Action')
        sumdf = tdf.groupby(['chain','project','action']).mean()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','action','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )
        print('Sum by App')
        sumdf = tdf.groupby(['chain','project']).sum()
        sumdf.reset_index(inplace=True)
        sumdf['usd_value_net_pct_share_100'] = sumdf['usd_value_net_pct_share'] * 100
        sumdf['usd_value_out_pct_share_100'] = sumdf['usd_value_out_pct_share'] * 100
        sumdf = sumdf.sort_values(by = 'usd_value_out_pct_share_100', ascending = False)
        sumdf = sumdf[['chain','project','usd_value_out_pct_share_100','usd_value_net_pct_share_100']]
        display( sumdf.head(10) )

Address-Weighted
Sum by Action


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/2832201673.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
7,Arbitrum,gmx,gmx - PositionRouter | 0xf2ae372f,51.880684,0.000000
15,Arbitrum,sushi,sushi - Pair | swapExactTokensForETH,43.063953,0.000000
13,Arbitrum,gmx,gmx - sGMX | 0xf3daeacc,35.380614,0.000000
3,Arbitrum,across_v2,across_v2 - Arbitrum_SpokePool | deposit,33.906252,0.000000
9,Arbitrum,gmx,gmx - Vault | 0x364e2311,31.299871,0.000000
18,Arbitrum,uniswap_v3,uniswap_v3 - Pair | mint,25.269913,0.000000
19,Arbitrum,uniswap_v3,uniswap_v3 - Pair | multicall,23.965963,20.857980
5,Arbitrum,celer_network,celer_network - Bridge | send,17.962489,0.000000
11,Arbitrum,gmx,gmx - Vault | swap,15.316558,0.075303
20,Arbitrum,uniswap_v3,uniswap_v3 - Pair | uniswapV3Swap,13.697262,9.424339


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/2832201673.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


...

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/2832201673.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project','action']).mean()


,chain,project,action,usd_value_out_pct_share_100,usd_value_net_pct_share_100
10,Optimism,Velodrome,Velodrome - Pair | addLiquidity,22.228758,0.000000
2,Optimism,Aave,Aave - AToken | repay,20.102279,0.000000
6,Optimism,Celer,Celer - Bridge | send,19.440471,0.000000
7,Optimism,OTHER,OTHER - OTHER | OTHER,11.095232,11.686569
3,Optimism,Aave,Aave - AToken | supply,10.575471,0.000000
9,Optimism,Uniswap,Uniswap - Pool | multicall,8.024692,6.789021
0,Optimism,1inch,1inch - AggregationRouterV5 | swap,2.481089,16.355563
1,Optimism,Aave,Aave - AToken | borrow,0.000000,13.263009
4,Optimism,Aave,Aave - AToken | withdraw,0.000000,27.050186
5,Optimism,Celer,Celer - Bridge | relay,0.000000,14.671806


Sum by App


/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_8904/2832201673.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sumdf = tdf.groupby(['chain','project']).sum()


,chain,project,usd_value_out_pct_share_100,usd_value_net_pct_share_100
1,Optimism,Aave,30.677751,40.313195
6,Optimism,Velodrome,22.228758,0.000000
2,Optimism,Celer,19.440471,14.671806
3,Optimism,OTHER,11.095232,11.686569
5,Optimism,Uniswap,8.024692,6.789021
0,Optimism,1inch,2.481089,16.355563
4,Optimism,Stargate Finance,0.000000,18.531796
